# 03: Extracting data to different formats

In [ ]:
library(IRdisplay)
display_html('<iframe width="560" height="315" src="https://www.youtube.com/embed/9-EDaRQ8Aps?si=8AVrrzCQXntpG8ER" frameborder="0" allowfullscreen></iframe>')

In this notebook, we will look at how we can export data from a NetCDF file into a CSV or XLSX file.

Let's import some modules first

In [1]:
library(RNetCDF)
library(writexl)

In this notebook you'll find examples for 1D, 2D and 3D variables.

## Variables with 1 dimension

In this section, we will use some CTD data from the Nansen Legacy project. If you use these data, please cite them as recommended below:

*Elizabeth Jones (2022) CTD data from Nansen Legacy Cruise - Joint cruise 2-1 Staion: P1_NLEG01-1 https://doi.org/10.21335/NMDC-2085836005-P1_NLEG01-1*

Let's load in the data.

In [2]:
url <- 'https://opendap1.nodc.no/opendap/physics/point/cruise/nansen_legacy-single_profile/NMDC_Nansen-Legacy_PR_CT_58US_2021708/CTD_station_P1_NLEG01-1_-_Nansen_Legacy_Cruise_-_2021_Joint_Cruise_2-1.nc'
data <- open.nc(url)
print.nc(data)

netcdf classic {
dimensions:
	PRES = 320 ;
variables:
	NC_FLOAT PRES(PRES) ;
		NC_FLOAT PRES:_FillValue = NaN ;
		NC_CHAR PRES:axis = "Z" ;
		NC_CHAR PRES:uncertainty = " " ;
		NC_CHAR PRES:comment = " " ;
		NC_CHAR PRES:ancillary_variables = "PRES_QC" ;
		NC_CHAR PRES:standard_name = "sea_water_pressure" ;
		NC_CHAR PRES:long_name = "Sea pressure" ;
		NC_CHAR PRES:units = "dbar" ;
		NC_CHAR PRES:positive = "down" ;
		NC_CHAR PRES:data_mode = "D" ;
		NC_CHAR PRES:coverage_content_type = "physicalMeasurement" ;
	NC_SHORT PRES_QC(PRES) ;
		NC_SHORT PRES_QC:_FillValue = -127 ;
		NC_CHAR PRES_QC:conventions = "Copernicus Marine In Situ reference table 2" ;
		NC_SHORT PRES_QC:valid_min = 0 ;
		NC_SHORT PRES_QC:valid_max = 9 ;
		NC_SHORT PRES_QC:flag_values = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ;
		NC_CHAR PRES_QC:flag_meanings = "no_qc_performed good_data probably_good_data bad_data_that_are_potentially_correctable bad_data value_changed value_below_detection nominal_value interpolated_value missi

The data have 1 dimension, pressure. Let's say we want to extract the data from the *TEMP* and *PSAL* variables to a CSV or XLSX file. We first need to create a dataframe.

In [3]:
pres <- var.get.nc(data, 'PRES')
temp <- var.get.nc(data, 'TEMP')
psal <- var.get.nc(data, 'PSAL')
df <- data.frame(Pressure = pres, Temperature = temp, Salinity = psal)
head(df)

,Pressure,Temperature,Salinity
,<dbl>,<dbl>,<dbl>
1,1,3.735,34.254
2,2,3.738,34.260
3,3,3.739,34.261
4,4,3.741,34.267
5,5,3.736,34.259
6,6,3.737,34.260


We can write the data to a CSV or XLSX file

In [4]:
write_xlsx(df, '../data/exported_from_notebooks/ctd_data.xlsx')
write.csv(df, '../data/exported_from_notebooks/ctd_data.csv', row.names = FALSE)

## Variables with 2 dimensions

Let's look at some sea ice concentration data.

If you use these data, please cite them as recommended below:

*Lavergne, T.; Sørensen, A.; Tonboe, R.T.; Kreiner, M.; Saldo, R.; Birkedal, A.; Baordo, F.; Aspenes, T.; Eastwood, S. (2023): ESA Sea Ice Climate Change Initiative (Sea_Ice_cci): High(er) Resolution Sea Ice Concentration Climate Data Record Version 3 (SSM/I and SSMIS). NERC EDS Centre for Environmental Data Analysis, 17 May 2023. doi:10.5285/eade27004395466aaa006135e1b2ad1a. https://dx.doi.org/10.5285/eade27004395466aaa006135e1b2ad1a*

In [5]:
url <- 'https://dap.ceda.ac.uk/thredds/dodsC/neodc/esacci/sea_ice/data/sea_ice_concentration/L4/ssmi_ssmis/12.5km/v3.0/NH/2007/09/ESACCI-SEAICE-L4-SICONC-RE_SSMI_12.5kmEASE2-NH-20070915-fv3.0.nc'
data <- open.nc(url)
print.nc(data)

netcdf classic {
dimensions:
	time = UNLIMITED ; // (1 currently)
	nv = 2 ;
	xc = 864 ;
	yc = 864 ;
variables:
	NC_INT Lambert_Azimuthal_Grid ;
		NC_CHAR Lambert_Azimuthal_Grid:grid_mapping_name = "lambert_azimuthal_equal_area" ;
		NC_DOUBLE Lambert_Azimuthal_Grid:longitude_of_projection_origin = 0 ;
		NC_DOUBLE Lambert_Azimuthal_Grid:latitude_of_projection_origin = 90 ;
		NC_DOUBLE Lambert_Azimuthal_Grid:false_easting = 0 ;
		NC_DOUBLE Lambert_Azimuthal_Grid:false_northing = 0 ;
		NC_DOUBLE Lambert_Azimuthal_Grid:semi_major_axis = 6378137 ;
		NC_DOUBLE Lambert_Azimuthal_Grid:inverse_flattening = 298.257223563 ;
		NC_CHAR Lambert_Azimuthal_Grid:proj4_string = "+proj=laea +lon_0=0 +datum=WGS84 +ellps=WGS84 +lat_0=90.0" ;
	NC_DOUBLE time(time) ;
		NC_CHAR time:units = "seconds since 1978-01-01 00:00:00" ;
		NC_CHAR time:long_name = "reference time of product" ;
		NC_CHAR time:standard_name = "time" ;
		NC_CHAR time:axis = "T" ;
		NC_CHAR time:calendar = "standard" ;
		NC_CHAR time:bounds

The *ice_conc* variable has 3 dimensions; xc, yc and time. However, since there is only one time slice in this dataset, this is essentially a 2D variable.

Let's write the *ice_conc* variable to a dataframe alongside the *xc* and *yc* coordinate variables.

In [6]:
xc <- var.get.nc(data, 'xc')
yc <- var.get.nc(data, 'yc')
ice_conc <- var.get.nc(data, 'ice_conc')
dim(ice_conc) # Showing that this is 2D, not 3D.

[1] 864 864

The order of the dimensions is important. *xc* and then *yc*. 

So the values are sorted such that all the values cooresponding to the first *xc* coordinate are first, before moving onto the second *xc* coordinate. 

Now let's create a 1D vector of the *ice_conc* data.

In [7]:
ice_conc_vector <- as.vector(ice_conc)

We then need to extend the *xc* coordinate values by repeating the first *xc* value 864 times before moving onto the second value. 

In [8]:
xc_repeat <- rep(xc, each = length(yc))
xc_repeat

[1] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
    [9] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
   [17] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
   [25] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
   [33] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
   [41] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
   [49] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
   [57] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
   [65] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
   [73] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
   [81] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
   [89] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
   [97] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [105] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [113] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [121] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [129] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [137] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [145] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [153] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [161] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [169] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [177] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [185] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [193] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [201] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [209] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [217] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [225] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [233] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [241] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [249] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [257] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [265] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [273] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [281] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [289] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [297] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [305] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [313] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [321] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [329] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [337] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [345] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [353] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [361] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [369] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [377] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [385] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [393] -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75 -5393.75
  [4

For the *yc* coordinate, we need to repeat the whole list 864 times.

In [9]:
yc_repeat <- rep(yc, times = length(xc))
yc_repeat

[1]  5393.75  5381.25  5368.75  5356.25  5343.75  5331.25  5318.75  5306.25
    [9]  5293.75  5281.25  5268.75  5256.25  5243.75  5231.25  5218.75  5206.25
   [17]  5193.75  5181.25  5168.75  5156.25  5143.75  5131.25  5118.75  5106.25
   [25]  5093.75  5081.25  5068.75  5056.25  5043.75  5031.25  5018.75  5006.25
   [33]  4993.75  4981.25  4968.75  4956.25  4943.75  4931.25  4918.75  4906.25
   [41]  4893.75  4881.25  4868.75  4856.25  4843.75  4831.25  4818.75  4806.25
   [49]  4793.75  4781.25  4768.75  4756.25  4743.75  4731.25  4718.75  4706.25
   [57]  4693.75  4681.25  4668.75  4656.25  4643.75  4631.25  4618.75  4606.25
   [65]  4593.75  4581.25  4568.75  4556.25  4543.75  4531.25  4518.75  4506.25
   [73]  4493.75  4481.25  4468.75  4456.25  4443.75  4431.25  4418.75  4406.25
   [81]  4393.75  4381.25  4368.75  4356.25  4343.75  4331.25  4318.75  4306.25
   [89]  4293.75  4281.25  4268.75  4256.25  4243.75  4231.25  4218.75  4206.25
   [97]  4193.75  4181.25  4168.75  4156.25  4143.75  4131.25  4118.75  4106.25
  [105]  4093.75  4081.25  4068.75  4056.25  4043.75  4031.25  4018.75  4006.25
  [113]  3993.75  3981.25  3968.75  3956.25  3943.75  3931.25  3918.75  3906.25
  [121]  3893.75  3881.25  3868.75  3856.25  3843.75  3831.25  3818.75  3806.25
  [129]  3793.75  3781.25  3768.75  3756.25  3743.75  3731.25  3718.75  3706.25
  [137]  3693.75  3681.25  3668.75  3656.25  3643.75  3631.25  3618.75  3606.25
  [145]  3593.75  3581.25  3568.75  3556.25  3543.75  3531.25  3518.75  3506.25
  [153]  3493.75  3481.25  3468.75  3456.25  3443.75  3431.25  3418.75  3406.25
  [161]  3393.75  3381.25  3368.75  3356.25  3343.75  3331.25  3318.75  3306.25
  [169]  3293.75  3281.25  3268.75  3256.25  3243.75  3231.25  3218.75  3206.25
  [177]  3193.75  3181.25  3168.75  3156.25  3143.75  3131.25  3118.75  3106.25
  [185]  3093.75  3081.25  3068.75  3056.25  3043.75  3031.25  3018.75  3006.25
  [193]  2993.75  2981.25  2968.75  2956.25  2943.75  2931.25  2918.75  2906.25
  [201]  2893.75  2881.25  2868.75  2856.25  2843.75  2831.25  2818.75  2806.25
  [209]  2793.75  2781.25  2768.75  2756.25  2743.75  2731.25  2718.75  2706.25
  [217]  2693.75  2681.25  2668.75  2656.25  2643.75  2631.25  2618.75  2606.25
  [225]  2593.75  2581.25  2568.75  2556.25  2543.75  2531.25  2518.75  2506.25
  [233]  2493.75  2481.25  2468.75  2456.25  2443.75  2431.25  2418.75  2406.25
  [241]  2393.75  2381.25  2368.75  2356.25  2343.75  2331.25  2318.75  2306.25
  [249]  2293.75  2281.25  2268.75  2256.25  2243.75  2231.25  2218.75  2206.25
  [257]  2193.75  2181.25  2168.75  2156.25  2143.75  2131.25  2118.75  2106.25
  [265]  2093.75  2081.25  2068.75  2056.25  2043.75  2031.25  2018.75  2006.25
  [273]  1993.75  1981.25  1968.75  1956.25  1943.75  1931.25  1918.75  1906.25
  [281]  1893.75  1881.25  1868.75  1856.25  1843.75  1831.25  1818.75  1806.25
  [289]  1793.75  1781.25  1768.75  1756.25  1743.75  1731.25  1718.75  1706.25
  [297]  1693.75  1681.25  1668.75  1656.25  1643.75  1631.25  1618.75  1606.25
  [305]  1593.75  1581.25  1568.75  1556.25  1543.75  1531.25  1518.75  1506.25
  [313]  1493.75  1481.25  1468.75  1456.25  1443.75  1431.25  1418.75  1406.25
  [321]  1393.75  1381.25  1368.75  1356.25  1343.75  1331.25  1318.75  1306.25
  [329]  1293.75  1281.25  1268.75  1256.25  1243.75  1231.25  1218.75  1206.25
  [337]  1193.75  1181.25  1168.75  1156.25  1143.75  1131.25  1118.75  1106.25
  [345]  1093.75  1081.25  1068.75  1056.25  1043.75  1031.25  1018.75  1006.25
  [353]   993.75   981.25   968.75   956.25   943.75   931.25   918.75   906.25
  [361]   893.75   881.25   868.75   856.25   843.75   831.25   818.75   806.25
  [369]   793.75   781.25   768.75   756.25   743.75   731.25   718.75   706.25
  [377]   693.75   681.25   668.75   656.25   643.75   631.25   618.75   606.25
  [385]   593.75   581.25   568.75   556.25   543.75   531.25   518.75   506.25
  [393]   493.75   481.25   468.75   456.25   443.75   431.25   418.75   406.25
  [4

And now let's combing them all in a dataframe and export the data.

In [10]:
df <- data.frame(
  Xc = xc_repeat,
  Yc = yc_repeat,
  SeaIceConcentration = ice_conc_vector
)
head(df)


,Xc,Yc,SeaIceConcentration
,<dbl>,<dbl>,<dbl>
1,-5393.75,5393.75,0
2,-5393.75,5381.25,0
3,-5393.75,5368.75,0
4,-5393.75,5356.25,0
5,-5393.75,5343.75,0
6,-5393.75,5331.25,0


Let's add the latitude and longitude values as separate columns.

In [11]:
latitude <- var.get.nc(data, "lat")
latitude_vector <- as.vector(latitude)
df$latitude <- latitude_vector

longitude <- var.get.nc(data, "lon")
longitude_vector <- as.vector(longitude)
df$longitude <- longitude_vector

head(df)
write_xlsx(df, '../data/exported_from_notebooks/2d_data.xlsx')
write.csv(df, '../data/exported_from_notebooks/2d_data.csv', row.names = FALSE)

,Xc,Yc,SeaIceConcentration,latitude,longitude
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-5393.75,5393.75,0,16.52435,-135.0000
2,-5393.75,5381.25,0,16.62387,-135.0665
3,-5393.75,5368.75,0,16.72321,-135.1331
4,-5393.75,5356.25,0,16.82236,-135.1999
5,-5393.75,5343.75,0,16.92134,-135.2668
6,-5393.75,5331.25,0,17.02014,-135.3339


# Variables with 3 dimensions

Let's look at the same data we used in [tutorial #01](01_opening_and_understanding.ipynb).

If you use these data, please cite them as recommended below:

*H.-M. Zhang, B. Huang, J. H. Lawrimore, M. J. Menne, and T. M. Smith (2019): NOAA Global Surface Temperature Dataset (NOAAGlobalTemp), Version 5.0. NOAA National Centers for Environmental Information. doi:10.25921/9qth-2p70 Accessed 2024-04-09.*

Let's load in the data, and extract the data from the *anom* varaible.

In [12]:
url <- 'https://www.ncei.noaa.gov/thredds/dodsC/noaa-global-temp-v5/NOAAGlobalTemp_v5.0.0_gridded_s188001_e202212_c20230108T133308.nc'
data <- open.nc(url)
print.nc(data)

netcdf classic {
dimensions:
	lat = 36 ;
	lon = 72 ;
	time = 1716 ;
	z = 1 ;
variables:
	NC_FLOAT time(time) ;
		NC_CHAR time:long_name = "reference time of global temperature anomalies" ;
		NC_CHAR time:standard_name = "time" ;
		NC_CHAR time:coverage_content_type = "coordinate" ;
		NC_CHAR time:units = "days since 1800-01-01 00:00:00" ;
		NC_CHAR time:calendar = "gregorian" ;
		NC_CHAR time:axis = "T" ;
	NC_FLOAT lat(lat) ;
		NC_CHAR lat:long_name = "Latitude" ;
		NC_CHAR lat:standard_name = "latitude" ;
		NC_CHAR lat:coverage_content_type = "coordinate" ;
		NC_CHAR lat:units = "degrees_north" ;
		NC_CHAR lat:grids = "Uniform grid from -87.5 to 87.5 by 5" ;
		NC_FLOAT lat:valid_min = -87.5 ;
		NC_FLOAT lat:valid_max = 87.5 ;
		NC_CHAR lat:axis = "Y" ;
		NC_CHAR lat:_CoordinateAxisType = "Lat" ;
		NC_CHAR lat:coordinate_defines = "center" ;
	NC_FLOAT lon(lon) ;
		NC_CHAR lon:long_name = "Longitude" ;
		NC_CHAR lon:standard_name = "longitude" ;
		NC_CHAR lon:coverage_content_type = "co

The anom variable actually has 4 dimensions (lon, lat, z and time). However, the z dimension is equal to 1, which means all the values are from the same elevation. Therefore, the data are essentially  3D.

Let's start by extracting the latitude, longitude and time variables as 1D vectors, and converting the time values to timestamps.

In [13]:
lat <- var.get.nc(data, 'lat')
lon <- var.get.nc(data, 'lon')
time <- var.get.nc(data, "time")

# Convert time to readable dates
start_date <- as.Date("1800-01-01")
timestamps <- as.Date(time, origin = start_date)

Now let's extract the temperature anomaly values. We can convert the 3D matrix into a 1D vector.

In [14]:
anom <- var.get.nc(data, 'anom')
anom_vector <- as.vector(anom)

The order of the dimensions is now important. We have longitude, latitude and then time.

This means that the anomaly values are sorted such that the all the values from the first longitude come first.

The values are next sorted by common latitude, and finally by common time.

It is a 3D grid has 72 points of longitude, 36 points of latitude, and 1716 points in time.

In [15]:
dim(anom)

[1]   72   36 1716

So when flattening the first dimension, longitude, to a 1D vector, we need to repeat the first value 36 x 1716 times before moving onto the second value.

In [16]:
lon_repeat <- rep(lon, each = length(time) * length(lat))

To flatten the second dimension, latitude, to a 1D vector, we need repeat the first value 1716 times before moving on to the second value. Then, after we have reached the last latitude, we need to start from the beginning and repeat 72 times.

In [17]:
lat_repeat <- rep(rep(lat, each = length(time)), times = length(lon))

Finally, for the third dimension, time, we need to list all the timestamps in order first, and then repeat the list 36 x 72 times.

In [18]:
timestamps_repeat <- rep(timestamps, times = length(lon) * length(lat))

Finally, let's write the data to a dataframe and export them.

In [ ]:
df <- data.frame(
  Timestamp = timestamps_repeat,
  Latitude = lat_repeat,
  Longitude = lon_repeat,
  TemperatureAnomalies = anom_vector
)

head(df)
write.csv(df, '../data/exported_from_notebooks/3d_data.csv', row.names = FALSE)
# There are too many rows in this data frame to convert it to a XLSX file

,Timestamp,Latitude,Longitude,TemperatureAnomalies
,<date>,<dbl>,<dbl>,<dbl>
1,1880-01-01,-87.5,2.5,NA
2,1880-02-01,-87.5,2.5,NA
3,1880-03-01,-87.5,2.5,NA
4,1880-04-01,-87.5,2.5,NA
5,1880-05-01,-87.5,2.5,NA
6,1880-06-01,-87.5,2.5,NA


## How to cite this course

If you think this course contributed to the work you are doing, consider citing it in your list of references. Here is a recommended citation:

Marsden, L. (2024, May 31). NetCDF in R - from beginner to pro. Zenodo. https://doi.org/10.5281/zenodo.11400754

And you can navigate to the publication and export the citation in different styles and formats by clicking the icon below.

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.11400754.svg)](https://doi.org/10.5281/zenodo.11400754)